# The Final Round

We can finally solve again the problem, with the new networks:

In [ ]:
%matplotlib inline
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import importlib

# Dynamically add the Empirical Model Learning folder to the python path
eml_path = '..'
if not eml_path in sys.path:
    sys.path.insert(1, eml_path)
    
# Load the data
data_fname = os.path.join('shared', 'za_processed.h5')
with pd.HDFStore(data_fname) as store:
    data = store['data']
    means_in = store['means_in']
    stds_in = store['stds_in']
    sim_in = store['sim_in']
    sim_out = store['sim_out']
    in_defaults = store['in_defaults']
    pop_size = store['meta']['pop_size']

    
# Read the NN model
from keras.models import model_from_json

# Load the model architecture
net_prefixes = ['nn_reg_{}'.format(t) for t in sim_out]
knet = {}
def load_keras_nets(knet):
    # Load scalar output NNs
    for target, net_prefix in zip(sim_out, net_prefixes):
        net_fname = os.path.join('shared', '{}.json'.format(net_prefix))
        with open(net_fname) as fp:
            knet[target] = model_from_json(fp.read())

        # Load the model weights
        wgt_fname = os.path.join('shared', '{}.h5'.format(net_prefix))
        knet[target].load_weights(wgt_fname)

    # Load vector output NN (this one is available in a single version)
    net_fname = os.path.join('shared', 'nn_reg.json')
    with open(net_fname) as fp:
        knet['all'] = model_from_json(fp.read())

    # Load the model weights
    wgt_fname = os.path.join('shared', 'nn_reg.h5')
    knet['all'].load_weights(wgt_fname)
        
load_keras_nets(knet)


from eml.net import describe as ndescribe
from eml.net.reader import keras_reader

# Convert the Keras models in the EML format
net = {}
def convert_keras_net(knet, net):
    # Convert scalar-output NNs
    for target in sim_out:
        net[target] = keras_reader.read_keras_sequential(knet[target])
    
    # Convert vector-output NN
    net['all'] = keras_reader.read_keras_sequential(knet['all'])
        
convert_keras_net(knet, net)

We manage the neuron bounds for the network encoding:

In [ ]:
# Obtain basic input bounds from out dataset

# Compute minima and maxima
X_min, X_max = data[sim_in].min(), data[sim_in].max()
# Standardize
X_min = (X_min - means_in) / stds_in
X_max = (X_max - means_in) / stds_in


# Compute basic bounds
from eml.net import process as nprocess

def net_basic_bounds(net):
    for target in list(sim_out) + ['all']:
        # Reset existing bounds (just to ensure idempotence)
        net[target].reset_bounds()

        # Enforce basic input bounds
        in_layer = net[target].layer(0)
        for neuron, lb, ub in zip(in_layer.neurons(), X_min, X_max):
            neuron.update_lb(lb)
            neuron.update_ub(ub)

        # Compute bounds for the hidden neurons via Interval Based Reasoning
        nprocess.ibr_bounds(net[target])

net_basic_bounds(net)

#
# Preprocessing: bound tightening
#
from eml.backend import cplex_backend

# Build a backend object
bkd = cplex_backend.CplexBackend()

# Run forward bound tightening
timelimit = 10
def net_bound_tightening(net, timelimit):
    for target in list(sim_out) + ['all']:
        nprocess.fwd_bound_tighthening(bkd, net=net[target],
                                      timelimit=timelimit)
        # Display the bounds
        print('=== TARGET: {}'.format(target))
        print(net[target])
        print()

net_bound_tightening(net, timelimit)

And finally we model and solve the overall problem:

In [ ]:
#
# Load problem data
#

import json

# Load the data about available measures
measure_fname = os.path.join('data', 'measures.json')
with open(measure_fname) as fp:
    mdata = json.load(fp)

# Separate the measure effect data from the combinations
effects = mdata['effects']
combinations = mdata['combinations']

#
# Build and solve a CPLEX model
#

from eml.backend import cplex_backend
import docplex.mp.model as cpx
from eml.net import embed as nembed
from eml import util

# Build a backend object
bkd = cplex_backend.CplexBackend()
# Build a docplex model
mdl = cpx.Model()
X_vars = []
Y_vars = []
Y_nat = []
X_nat = []
M_vars = []
C_vars = []
M_map = {}
cost_var = mdl.continuous_var(name='cost')

# Model construction functions
def build_inout_vars(bkd, mdl, X_vars, Y_vars):
    # Build one variable for each network input
    for in_name, lb, ub in zip(sim_in, X_min, X_max):
        X_vars.append(mdl.continuous_var(lb=lb, ub=ub, name=in_name))

    # Build one variable for each network output
    for out_name in sim_out:
        # NOTE use slightly larger bounds (to account for approximation errors)
        Y_vars.append(mdl.continuous_var(lb=-1, ub=2, name=out_name))


def build_nat_out(bkd, mdl, Y_nat):
    for i, out_name in enumerate(sim_out):
        # Build the natural scale variable
        lb = Y_vars[i].lb * pop_size
        ub = Y_vars[i].ub * pop_size
        ynat = mdl.continuous_var(lb=lb, ub=ub, name=out_name+'_nat')
        Y_nat.append(ynat)

        # Add the standardization constraints
        mdl.add_constraint(Y_nat[i] == Y_vars[i] * pop_size)       


def build_nat_in(bkd, mdl, X_nat):
    for i, (in_name, lb, ub) in enumerate(zip(sim_in, X_min, X_max)):
        # Build the natural scale variable
        mean, std = means_in[in_name], stds_in[in_name]
        lb_nat = lb * std + mean
        ub_nat = ub * std + mean
        span = ub_nat - lb_nat
        xnat = mdl.continuous_var(lb=lb_nat-span, ub=ub_nat+span,
                                  name=in_name+'_nat')
        X_nat.append(xnat)

        # Add the capping & standardization constraints
        nat_nodes = [lb_nat-span, lb_nat, ub_nat, ub_nat+span]
        std_nodes = [lb, lb, ub, ub]
        util.encode_pwl(bkd, mdl, xvars=[X_nat[i], X_vars[i]],
                        nodes=[nat_nodes, std_nodes],
                       name=in_name)

        
def build_measure_vars(bkd, mdl, M_vars, C_vars, M_map):
    # Build one binary variable per measure
    for i, effect in enumerate(effects):
        mvar = mdl.binary_var(name=effect['name'])
        M_vars.append(mvar)
        M_map[effect['name']] = i

    # Build one binary variable per combination
    for i, combo in enumerate(combinations):
        cvar = mdl.binary_var(name='-'.join(combo['deps']))
        C_vars.append(cvar)        


def build_dependencies(bkd, mdl):
    for i, combo in enumerate(combinations):
        ndeps = len(combo['deps'])
        mvars = [M_vars[M_map[name]] for name in combo['deps']]
        mdl.add_constraint(ndeps * C_vars[i] <= sum(mvars))


def build_measure_effect_csts(bkd, mdl):
    for i, in_name in enumerate(sim_in):
        # Effects to input
        coefs, evars = [], []
        for j, effect in enumerate(effects):
            if in_name in effect:
                coefs.append(effect[in_name])
                evars.append(M_vars[j])
        # Combinations to input
        for j, combo in enumerate(combinations):
            if in_name in combo:
                coefs.append(combo[in_name])
                evars.append(C_vars[j])
        # Build the connection constraint
        mdl.add_constraint(X_nat[i] == in_defaults[i] + mdl.scal_prod(evars, coefs))


def build_cost_structure(bkd, mdl, cost_var):
    # Measures to cost
    coefs, evars = [], []
    for i, effect in enumerate(effects):
        coefs.append(effect['cost'])
        evars.append(M_vars[i])
    mdl.add_constraint(cost_var == mdl.scal_prod(evars, coefs))


def build_network_encoding(bkd, mdl, mode='scalar'):
    if mode == 'scalar':
        for i, target in enumerate(sim_out):
            nembed.encode(bkd, net[target], mdl, X_vars, Y_vars[i], target)
    elif mode == 'vector':
        nembed.encode(bkd, net['all'], mdl, X_vars, Y_vars, 'all')

        
# Build input and output variables
build_inout_vars(bkd, mdl, X_vars, Y_vars)
# Build a natural scale version of each output variable
build_nat_out(bkd, mdl, Y_nat)
# Build a natural scale version of each input variable
build_nat_in(bkd, mdl, X_nat)
# Build variables for the measures and their combinations
build_measure_vars(bkd, mdl, M_vars, C_vars, M_map)
# Build dependency constraints
build_dependencies(bkd, mdl)
# Connect effects and combinations to the natural-scale network inputs
build_measure_effect_csts(bkd, mdl)
# Build and constraint the cost variable
build_cost_structure(bkd, mdl, cost_var)
# Encode the network
build_network_encoding(bkd, mdl, 'vector')

# Budget
mdl.add_constraint(cost_var <= 70)
# Zombies
mdl.add_constraint(Y_nat[0] <= 20)
# Survivors
mdl.set_objective('max', Y_nat[0])

# Solve
mdl.set_time_limit(30)
print('=== Starting the solution process')
sol = mdl.solve()

if sol is None:
    print('No solution found')
else:
    print('=== SOLUTION DATA')
    print('Solution time: {:.2f} (sec)'.format(mdl.solve_details.time))
    print('Solver status: {}'.format(sol.solve_details.status))
    print('Survivors: {}'.format(sol[Y_nat[1]]))
    print('Zombies: {}'.format(sol[Y_nat[0]]))
    print('Cost: {}'.format(sol[cost_var]))
    print('Chosen measures:')
    for x, effect in zip(M_vars, effects):
        if sol[x] > 0:
            print('* {}'.format(effect['name']))
    print('Applicable bonuses:')
    for x, combo in zip(C_vars, combinations):
        if sol[x] > 0:
            print('* {}'.format(' + '.join(combo['deps'])))
    cstring = 'c({})'.format(', '.join('{:.3f}'.format(max(0, sol[x])) for x in X_nat))
    print('Evaluation string: {}'.format(cstring))

Our old solution is no longer viable: we need to increase the budget if we want to keep things under control.

We can now evaluate this final solution.